# Vertex Matching Engine GRPC: Batch Get Embeddings by ID

For PSA and PSC methods of Matching Engine (following code does not support public method)

## Setup

Pip install required packages

In [ ]:
! pip install google-cloud-aiplatform grpcio grpcio-tools

Create directory for required packages and proto specification

In [2]:
! mkdir third_party

In [1]:
%cd third_party

/home/jupyter/third_party


Clone repo with google API dependencies

In [5]:
! git clone https://github.com/googleapis/googleapis.git

Cloning into 'googleapis'...
remote: Enumerating objects: 210572, done.
remote: Counting objects: 100% (11364/11364), done.
remote: Compressing objects: 100% (580/580), done.
remote: Total 210572 (delta 10847), reused 11002 (delta 10773), pack-reused 199208
Receiving objects: 100% (210572/210572), 187.03 MiB | 41.89 MiB/s, done.
Resolving deltas: 100% (179875/179875), done.


Wget the matching engine proto file

In [8]:
! wget https://raw.githubusercontent.com/googleapis/python-aiplatform/main/google/cloud/aiplatform/matching_engine/_protos/match_service.proto

--2023-07-31 22:32:26--  https://raw.githubusercontent.com/googleapis/python-aiplatform/main/google/cloud/aiplatform/matching_engine/_protos/match_service.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7203 (7.0K) [text/plain]
Saving to: ‘match_service.proto’

match_service.proto 100%[===================>]   7.03K  --.-KB/s    in 0.001s  

2023-07-31 22:32:26 (10.8 MB/s) - ‘match_service.proto’ saved [7203/7203]



See proto file for request parameters

In [11]:
! cat match_service.proto

syntax = "proto3";

package google.cloud.aiplatform.container.v1;

import "google/rpc/status.proto";

// MatchService is a Google managed service for efficient vector similarity
// search at scale.
service MatchService {
  // Returns the nearest neighbors for the query. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc Match(MatchRequest) returns (MatchResponse) {}

  // Returns the nearest neighbors for batch queries. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc BatchMatch(BatchMatchRequest) returns (BatchMatchResponse) {}

  // Looks up the embeddings.
  rpc BatchGetEmbeddings(BatchGetEmbeddingsRequest)
      returns (BatchGetEmbeddingsResponse) {}
}

// Parameters for a match query.
message MatchRequest {
  // The ID of the DeploydIndex that will serve the request.
  // This MatchRequest is sent to a specific IndexEndpoint of the Control API,
  // as per the IndexEndpoint.network. That Ind

Compile the proto file

In [2]:
! python -m grpc_tools.protoc -I=. --proto_path=./googleapis --python_out=. --grpc_python_out=. match_service.proto

/opt/conda/envs/vme/lib/python3.9/site-packages/grpc_tools/protoc.py:21: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## Required Input

In [ ]:
# Set IP address for your matching engine index deployment (if using PSA, you can find the IP in the console UI under Vertex AI Matching Engine)
ENDPT_ID=""
# Set deployed index ID (can also be found in the console UI)
DEPLOYED_INDEX_ID=""
# List of your embedding IDs
EMBED_IDS=[...]

In [55]:
# # Set IP address for your matching engine index deployment (if using PSA, you can find the IP in the console UI under Vertex AI Matching Engine)
# ENDPT_ID="10.128.15.225"
# # Set deployed index ID (can also be found in the console UI)
# DEPLOYED_INDEX_ID="langchain_vector_store_20230622195444"
# # List of your embedding IDs
# EMBED_IDS=! gsutil ls gs://axel-argolis-usc1-bucket/documents | grep -o -P "(?<=gs:\/\/axel-argolis-usc1-bucket\/documents\/).*"
# EMBED_IDS=EMBED_IDS[:3]

# Batch Get Embeddings from Matching Engine Index Endpoint

Import grpc and the compiled packages from the proto file

In [56]:
import grpc
import match_service_pb2
import match_service_pb2_grpc

In [57]:
# Set up channel and stub
channel = grpc.insecure_channel("{}:10000".format(ENDPT_ID))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

request = match_service_pb2.BatchGetEmbeddingsRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for id in EMBED_IDS:
    request.id.append(id)

response = stub.BatchGetEmbeddings(request)
response

embeddings {
  id: "01da2a71-18e0-4a46-93ba-b8909c2aec02"
  float_val: -0.0189142283
  float_val: -0.0471350327
  float_val: 0.0208867472
  float_val: 0.0400176682
  float_val: 0.025287563
  float_val: -0.0254594777
  float_val: 0.0156285204
  float_val: -0.0440849625
  float_val: -0.0360203385
  float_val: 0.00638740789
  float_val: 0.00984383095
  float_val: -0.0147382449
  float_val: 0.0379027762
  float_val: 0.0315231681
  float_val: -0.0339911245
  float_val: 0.000505836913
  float_val: -0.0247009
  float_val: 0.0198730659
  float_val: 0.0524923168
  float_val: 0.0161992908
  float_val: -0.0509170815
  float_val: -0.0323860198
  float_val: 0.02487
  float_val: -0.00328659941
  float_val: -0.00494350353
  float_val: -0.0789920092
  float_val: 0.00118659262
  float_val: 0.0483987294
  float_val: 0.00813204795
  float_val: 0.0537335798
  float_val: -0.0198349077
  float_val: -0.00858043507
  float_val: -0.0485354066
  float_val: -0.0265092365
  float_val: -6.90619781e-05
  float_val:

Example of a single embedding:

In [48]:
response.embeddings[0].float_val

[-0.02219032309949398, -0.032976604998111725, -0.009809693321585655, 0.0014791799476370215, 0.023153193295001984, -0.018439648672938347, 0.0036928679328411818, 0.004296474624425173, 0.007074352819472551, 0.04393679276108742, -0.000595985387917608, 0.008479141630232334, 0.009828307665884495, -0.0002847194846253842, -0.036049578338861465, -0.0028231218457221985, -0.08940312266349792, -0.016170000657439232, 0.05688387528061867, 0.0015358756063506007, -0.02112380415201187, -0.03614622727036476, -0.0013125981204211712, -0.006161356810480356, -0.03780196234583855, -0.09045115858316422, 0.0280845258384943, 0.019154110923409462, -0.007887337356805801, 0.0017468787264078856, 0.029022149741649628, 0.0028538862243294716, -0.037322234362363815, -0.007214528974145651, 0.03648856654763222, -0.002990931738168001, 0.0064543443731963634, 0.03430083394050598, 0.013198553584516048, 0.010635798797011375, 0.050070565193891525, 0.03121698647737503, 0.04227771982550621, 0.005237790290266275, 0.01083069853484